In [1]:
import tensorflow as tf
import keras 
from keras.applications import VGG16, DenseNet121, DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

Using TensorFlow backend.


In [2]:
conv_base1 = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))
conv_base2 = DenseNet121(weights='imagenet', include_top = False, input_shape=(300,300,3))
conv_base3 = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [4]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [4]:
conv_base1.predict(X[0].reshape(1,300,300,3)).shape, conv_base2.predict(X[0].reshape(1,300,300,3)).shape, conv_base3.predict(X[0].reshape(1,300,300,3)).shape

((1, 9, 9, 512), (1, 9, 9, 1024), (1, 9, 9, 1920))

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [5]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [4]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [8]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=nos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


In [9]:
for samples_batch,labels_batch in numit:
    print(samples_batch.shape)
    X[::] = samples_batch
    break

(472, 300, 300, 3)


In [10]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [11]:
accuracies = []
f = 1

for train_index, test_index in skf.split(X, y):
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]
      
        X_train1 = conv_base1.predict(X_train)
        X_train1 = X_train1.reshape(X_train.shape[0],9*9*512)
        X_test1 = conv_base1.predict(X_test)
        X_test1 = X_test1.reshape(X_test.shape[0],9*9*512)
        
        X_train2 = conv_base2.predict(X_train)
        X_train2 = X_train2.reshape(X_train.shape[0],9*9*1024)
        X_test2 = conv_base2.predict(X_test)
        X_test2 = X_test2.reshape(X_test.shape[0],9*9*1024)
        
        X_train3 = conv_base3.predict(X_train)
        X_train3 = X_train3.reshape(X_train.shape[0],9*9*1920)
        X_test3 = conv_base3.predict(X_test)
        X_test3 = X_test3.reshape(X_test.shape[0],9*9*1920)

        model1 = models.Sequential()
        model1.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
        model1.add(layers.Dropout(0.5))
        model1.add(layers.Dense(1, activation = 'sigmoid'))
        model1.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        model2 = models.Sequential()
        model2.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
        model2.add(layers.Dropout(0.5))
        model2.add(layers.Dense(1, activation = 'sigmoid'))
        model2.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        model3 = models.Sequential()
        model3.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
        model3.add(layers.Dropout(0.5))
        model3.add(layers.Dense(1, activation = 'sigmoid'))
        model3.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        
        history1 = model1.fit(X_train1, y_train, epochs = 100, batch_size = 20, validation_data = (X_test1, y_test), shuffle = True, callbacks = my_callbacks)
        history2 = model2.fit(X_train2, y_train, epochs = 100, batch_size = 20, validation_data = (X_test2, y_test), shuffle = True, callbacks = my_callbacks)
        history3 = model3.fit(X_train3, y_train, epochs = 100, batch_size = 20, validation_data = (X_test3, y_test), shuffle = True, callbacks = my_callbacks)


        test_preds1 = model1.predict(X_test1, batch_size = 20)
        test_predclass1 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds1])

        test_preds2 = model2.predict(X_test2, batch_size = 20)
        test_predclass2 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds2])

        test_preds3 = model3.predict(X_test3, batch_size = 20)
        test_predclass3 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds3])

        kfinal_preds = np.empty(test_predclass3.shape[0])
        for i in range(test_predclass3.shape[0]):
          predsum = test_predclass1[i] + test_predclass2[i] + test_predclass3[i] 
          kfinal_preds[i] = 0.0 if predsum in [0.0,1.0] else 1.0


        accuracies.append([confusion_matrix(y_test,kfinal_preds)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,kfinal_preds)[1][1]/(y_test == 1.0).sum()])
        f+=1

Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 0s 1ms/step - loss: 8.1482 - accuracy: 0.6804 - val_loss: 4.0400 - val_accuracy: 0.6271
Epoch 2/100
413/413 [==============================] - 0s 799us/step - loss: 4.0123 - accuracy: 0.7700 - val_loss: 3.9942 - val_accuracy: 0.6441
Epoch 3/100
413/413 [==============================] - 0s 781us/step - loss: 2.1597 - accuracy: 0.8257 - val_loss: 2.5865 - val_accuracy: 0.6610
Epoch 4/100
413/413 [==============================] - 0s 791us/step - loss: 0.6790 - accuracy: 0.8668 - val_loss: 2.1082 - val_accuracy: 0.7288
Epoch 5/100
413/413 [==============================] - 0s 788us/step - loss: 0.5799 - accuracy: 0.8741 - val_loss: 1.2125 - val_accuracy: 0.6441
Epoch 6/100
413/413 [==============================] - 0s 769us/step - loss: 0.4029 - accuracy: 0.9249 - val_loss: 1.2402 - val_accuracy: 0.7458
Epoch 7/100
413/413 [==============================] - 0s 777us/step - loss: 0.4

In [12]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.758477011494253 0.725


Training on whole Training Dataset

In [13]:
conv1_samples = np.zeros(shape=(nos,9,9,512))
conv1_labels = np.zeros(shape=(nos))

for samples_batch,labels_batch in numit:
    f = conv_base1.predict(samples_batch)
    print(f.shape)
    conv1_samples = f
    conv1_labels = labels_batch
    break

(472, 9, 9, 512)


In [14]:
conv1_samples = conv1_samples.reshape(nos,9*9*512)

In [15]:
conv2_samples = np.zeros(shape=(nos,9,9,1024))
conv2_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base2.predict(samples_batch)
    print(f.shape)
    conv2_samples = f
    conv2_labels = labels_batch
    break

(472, 9, 9, 1024)


In [16]:
conv2_samples = conv2_samples.reshape(nos,9*9*1024)

In [17]:
conv3_samples = np.zeros(shape=(nos,9,9,1920))
conv3_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base3.predict(samples_batch)
    print(f.shape)
    conv3_samples = f
    conv3_labels = labels_batch
    break

(472, 9, 9, 1920)


In [18]:
conv3_samples = conv3_samples.reshape(nos,9*9*1920)

In [16]:
conv1_samples.shape, conv1_labels.shape, conv2_samples.shape, conv2_labels.shape, conv3_samples.shape, conv3_labels.shape

((472, 41472), (472,), (472, 82944), (472,), (472, 155520), (472,))

In [19]:
model_conv1 = models.Sequential()
model_conv1.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model_conv1.add(layers.Dropout(0.5))
model_conv1.add(layers.Dense(1, activation = 'sigmoid'))
model_conv1.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

model_conv2 = models.Sequential()
model_conv2.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
model_conv2.add(layers.Dropout(0.5))
model_conv2.add(layers.Dense(1, activation = 'sigmoid'))
model_conv2.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

model_conv3 = models.Sequential()
model_conv3.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
model_conv3.add(layers.Dropout(0.5))
model_conv3.add(layers.Dense(1, activation = 'sigmoid'))
model_conv3.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

In [20]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [21]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)
datagentest.fit(xtest)
testnos = ytest.shape[0]


numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


In [22]:
conv1_test_samples = np.zeros(shape=(testnos,9,9,512))
conv1_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base1.predict(samples_batch)
    print(g.shape)
    conv1_test_samples[::] = g
    conv1_test_labels[:] = labels_batch
    break

(552, 9, 9, 512)


In [23]:
conv1_test_samples = conv1_test_samples.reshape(testnos,9*9*512)

In [24]:
conv2_test_samples = np.zeros(shape=(testnos,9,9,1024))
conv2_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base2.predict(samples_batch)
    print(g.shape)
    conv2_test_samples[::] = g
    conv2_test_labels[:] = labels_batch
    break

(552, 9, 9, 1024)


In [25]:
conv2_test_samples = conv2_test_samples.reshape(testnos,9*9*1024)

In [26]:
conv3_test_samples = np.zeros(shape=(testnos,9,9,1920))
conv3_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base3.predict(samples_batch)
    print(g.shape)
    conv3_test_samples[::] = g
    conv3_test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [27]:
conv3_test_samples = conv3_test_samples.reshape(testnos,9*9*1920)

In [28]:
# To save memory
del xtest
del ytest

In [29]:
conv1_historytest = model_conv1.fit(conv1_samples, conv1_labels, epochs = 100, batch_size = 20, validation_data = (conv1_test_samples, conv1_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 1ms/step - loss: 2.0956 - accuracy: 0.6356 - val_loss: 0.6302 - val_accuracy: 0.7138
Epoch 2/100
472/472 [==============================] - 0s 1ms/step - loss: 0.3950 - accuracy: 0.8284 - val_loss: 0.4339 - val_accuracy: 0.8043
Epoch 3/100
472/472 [==============================] - 0s 1ms/step - loss: 0.2448 - accuracy: 0.9025 - val_loss: 0.4822 - val_accuracy: 0.7717
Epoch 4/100
472/472 [==============================] - 0s 1ms/step - loss: 0.2097 - accuracy: 0.9195 - val_loss: 0.6165 - val_accuracy: 0.7029
Epoch 5/100
472/472 [==============================] - 0s 1ms/step - loss: 0.1561 - accuracy: 0.9407 - val_loss: 0.9289 - val_accuracy: 0.6250
Epoch 6/100
472/472 [==============================] - 0s 1ms/step - loss: 0.1531 - accuracy: 0.9492 - val_loss: 0.5310 - val_accuracy: 0.7699
Epoch 7/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1638 - accuracy: 0.

In [30]:
conv2_historytest = model_conv2.fit(conv2_samples, conv2_labels, epochs = 100, batch_size = 20, validation_data = (conv2_test_samples, conv2_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 2ms/step - loss: 9.3381 - accuracy: 0.6271 - val_loss: 2.2771 - val_accuracy: 0.6866
Epoch 2/100
472/472 [==============================] - 1s 2ms/step - loss: 1.8178 - accuracy: 0.7860 - val_loss: 1.3352 - val_accuracy: 0.7011
Epoch 3/100
472/472 [==============================] - 1s 2ms/step - loss: 0.4682 - accuracy: 0.8708 - val_loss: 0.7875 - val_accuracy: 0.7355
Epoch 4/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2834 - accuracy: 0.8941 - val_loss: 0.6164 - val_accuracy: 0.8261
Epoch 5/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2678 - accuracy: 0.9110 - val_loss: 0.8846 - val_accuracy: 0.6322
Epoch 6/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2290 - accuracy: 0.9280 - val_loss: 0.6594 - val_accuracy: 0.6975
Epoch 7/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1875 - accuracy: 0.

In [31]:
conv3_historytest = model_conv3.fit(conv3_samples, conv3_labels, epochs = 100, batch_size = 20, validation_data = (conv3_test_samples, conv3_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 2s 4ms/step - loss: 13.8379 - accuracy: 0.6229 - val_loss: 1.8439 - val_accuracy: 0.9058
Epoch 2/100
472/472 [==============================] - 2s 3ms/step - loss: 2.4283 - accuracy: 0.7564 - val_loss: 1.1544 - val_accuracy: 0.6667
Epoch 3/100
472/472 [==============================] - 2s 3ms/step - loss: 0.4206 - accuracy: 0.8496 - val_loss: 0.4013 - val_accuracy: 0.8279
Epoch 4/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2764 - accuracy: 0.8983 - val_loss: 0.8747 - val_accuracy: 0.6594
Epoch 5/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2884 - accuracy: 0.8919 - val_loss: 0.9916 - val_accuracy: 0.5851
Epoch 6/100
472/472 [==============================] - 2s 3ms/step - loss: 0.2193 - accuracy: 0.9174 - val_loss: 0.6407 - val_accuracy: 0.7337
Epoch 7/100
472/472 [==============================] - 2s 3ms/step - loss: 0.1856 - accuracy: 0

In [32]:
conv1_test_preds = model_conv1.predict(conv1_test_samples, batch_size = 20)
conv1_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv1_test_preds])

In [33]:
conv2_test_preds = model_conv2.predict(conv2_test_samples, batch_size = 20)
conv2_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv2_test_preds])

In [34]:
conv3_test_preds = model_conv3.predict(conv3_test_samples, batch_size = 20)
conv3_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv3_test_preds])

In [35]:
# Voting

final_preds = np.empty(conv1_test_predclass.shape[0])
for i in range(conv1_test_predclass.shape[0]):
  predsum = conv1_test_predclass[i] + conv2_test_predclass[i] + conv3_test_predclass[i] 
  final_preds[i] = 0.0 if predsum in [0.0,1.0] else 1.0

In [36]:
confusion_matrix(conv1_test_labels,final_preds)[0][0]/(conv1_test_labels == 0.0).sum(), confusion_matrix(conv1_test_labels,final_preds)[1][1]/(conv1_test_labels == 1.0).sum()

(0.6610169491525424, 0.8701825557809331)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/